# Convert sensitivities and fluxdiagrams to ARC input format

This script aims to build a bridge between the RMG outputs and the ARC.<br>
1. It reads sensitivities generated by RMG sensitivity analysis, as well as fluxdiagrams generated by rms. 
2. based on certain critria, the script will figure out which species to be run. 
3. based on the species list, it will generate an ARC input format.

### Necessary packages

In [ ]:
import logging
import os

from rmgpy.chemkin import load_species_dictionary

from toolbox.base import get_files_by_regex, read_species_from_yml, write_species_to_yml
from toolbox.fluxdiagram import get_spc_label_from_fluxdiagrams
from toolbox.sensitivity import get_spc_label_from_sa
from toolbox.species import get_spc_list_from_labels, rmg_chemkin_label

%load_ext autoreload
%autoreload 2

logger = logging.getLogger()
logger.setLevel(logging.INFO)

###  [OPTIONAL] Assign a log file to record all the changes

In [ ]:
fh = logging.FileHandler('rop_sa_to_arc.log', mode="a+")
fh.setLevel(logging.INFO)
fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s: %(message)s'))
logger.addHandler(fh)

### [INPUT] Read molecule information
- `chem_path` (str): The path to your CHEMKIN FILE
- `dict_path` (str): The path to the species dictionary

In [1]:
chem_path = ''
dict_path = ''

In [ ]:
spc_dict = load_species_dictionary(dict_path)

### [INPUT] Flux diagrams to ARC input file
hint: You need to save your all flux diagram as `.dot`. It is automatically done by RMS or RMG
- `path` (str): The path contains all your flux diagrams
- `rop_path` (str): The path to save extracted molecule info from ROP analysis
- `generate_from` (str): If generate from RMG fluxdiagram module, then the labels used in diagram are RMG labels instead of CHEMKIN labels

In [ ]:
path = ''
save_path = ''
generate_from = 'rmg'

In [ ]:
fd_list = get_files_by_regex(path, r"^.+\.dot$")
label_list = get_spc_label_from_fluxdiagrams(fd_list)
if generate_from == 'rmg':
    rmg_to_chemkin_label, _ = rmg_chemkin_label(chem_path)
    label_list = [rmg_to_chemkin_label[label] for label in label_list]
spc_list = get_spc_list_from_labels(label_list, spc_dict)
write_species_to_yml(spc_list, save_path, mode='overwrite')

### [INPUT] Sensitivity analysis to ARC input file
- `path` (str): The path contains all sensitivity results
- `sa_path` (str): The path to save extracted molecule info from SA analysis
- `N` (int): the upperbound number of species to be extracted in each SA

In [ ]:
path = ''
save_path = ''
N = 50

In [ ]:
sa_list = get_files_by_regex(path, r"^sensitivity.+\.csv$")
label_list = get_spc_label_from_sa(sa_list)
spc_list = get_spc_list_from_labels(label_list, spc_dict)
write_species_to_yml(spc_list, sa_path, mode='overwrite') 

###  [INPUT] Combine species list from ROP and SA
- `rop_path` (str): The path to ARC input file generated from ROP analysis
- `sa_path` (str): The path to ARC input file generated from SA analysis
- `save_path` (str): The path to save combined species info

In [ ]:
rop_result = ''
sa_result = ''
save_path = ''

In [ ]:
rop_spc = read_species_from_yml(rop_result)
sa_spc = read_species_from_yml(sa_result)
spc_list = combine_spc_list(rop_spc, sa_spc)
write_species_to_yml(spc_list, save_path)